In [1]:
# Libraries
import os
import numpy as np
import keras
import tensorflow as tf
from importlib import reload

# Scripts
from helpers import data, utils

In [2]:
!pip show tensorflow

Name: tensorflow
Version: 2.10.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\sonar\anaconda3v2\envs\acml_2024\lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, keras-preprocessing, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: 


In [3]:
reload(data)
reload(utils)

<module 'helpers.utils' from 'c:\\Users\\sonar\\Desktop\\thesis-ml\\ml\\main\\helpers\\utils.py'>

In [4]:
# Silence information messages from TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

# Check whether tf is using the GPU
utils.check_gpu_use() 

GPU DETECTED ✓
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## `main`

In [5]:
NUM_EPOCHS = 1
GPU_BATCH_SIZE = 16

### Load data

Because the QiandaoEar22 subset is quite small, we can simply load it all into memory at once.

In [6]:
from tqdm import tqdm
import scipy

path = '../data/qiandaoear22_kaiyuan_mat'
sub_folders = ['target', 'other']

data_1D = []

for folder in sub_folders:
    subfolder_path = os.path.join(path, folder)
    for datafile in tqdm(os.listdir(subfolder_path)):
        datafile_path = os.path.join(subfolder_path, datafile)
        data_1D.append(scipy.io.loadmat(datafile_path)['Ptrans'])
       

100%|██████████| 4000/4000 [00:35<00:00, 112.66it/s]


In [7]:
specs = np.stack(data_1D)
specs = np.expand_dims(specs, axis=-1)
print(specs.shape)

del data_1D

(5947, 192, 192, 1)


### Initialise model

In [8]:
from models import unet_n2n
reload(unet_n2n)

model = unet_n2n.get_unet_model(input_shape=(192, 192, 1))
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5, beta_1=0.9, beta_2=0.99, epsilon=1e-8),
    loss=keras.losses.MeanSquaredError()
)
model.summary()

Model: "unet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 192, 192, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 192, 192, 64  640         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 192, 192, 64  0           ['conv2d[0][0]']                 
                                )                                                              

### Configure and fit model

In [9]:
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath="models/saved/qiandaoear_pretrain_12122024/irfan/{epoch:02d}.keras",
    monitor='loss',
    save_best_only=True
)

csv_logger = keras.callbacks.CSVLogger(
    filename="models/saved/qiandaoear_pretrain_12122024/irfan/training.log",
    separator=',',
    append=False
)

backup_callback = keras.callbacks.BackupAndRestore(
    backup_dir="models/saved/qiandaoear_pretrain_12122024/irfan/tmp"
)

In [10]:
# with tf.device('/CPU:0'):
history = model.fit(
    x=specs,
    y=specs,
    batch_size=GPU_BATCH_SIZE,
    epochs=NUM_EPOCHS,
    verbose=2
)

NotFoundError: Graph execution error:

Detected at node 'gradient_tape/unet/conv2d_16/Conv2D/Conv2DBackpropInput' defined at (most recent call last):
    File "c:\Users\sonar\anaconda3v2\envs\acml_2024\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\sonar\anaconda3v2\envs\acml_2024\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\sonar\anaconda3v2\envs\acml_2024\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\sonar\anaconda3v2\envs\acml_2024\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\sonar\anaconda3v2\envs\acml_2024\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\sonar\anaconda3v2\envs\acml_2024\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop

In [ ]:
model.save("models/saved/qiandaoear_pretrain_12122024/irfan/irfan_qiandao_10epochs.keras")

### Calculate metrics

In [ ]:
reload(utils)

fig = utils.get_loss_curve(history)
fig.show()